In [2]:
import cv2
import numpy as np

# ==== CONFIG ====
IMAGE_PATH = "doi_10_5061_dryad_5dv41nsfj__v20241013/images_original/1.3.1. original.jpg"

# โหลดภาพ
img = cv2.imread(IMAGE_PATH)
if img is None:
    raise FileNotFoundError(f"Cannot load {IMAGE_PATH}")

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 5)

# ==== 1. Detect candidate disks (ยาเม็ด) ====
circles = cv2.HoughCircles(
    gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=40,
    param1=50, param2=30, minRadius=20, maxRadius=60
)

if circles is not None:
    circles = np.round(circles[0, :]).astype("int")

    for (x, y, r) in circles:
        # สร้าง mask วงกลมสำหรับเช็คความสว่าง
        mask = np.zeros_like(gray)
        cv2.circle(mask, (x, y), r, 255, -1)
        mean_val = cv2.mean(gray, mask=mask)[0]

        # filter: เอาเฉพาะเม็ดยาที่ขาว
        if mean_val < 180:
            continue

        # วาดเม็ดยา
        cv2.circle(img, (x, y), r, (0, 255, 0), 2)
        cv2.circle(img, (x, y), 2, (0, 0, 255), 3)

        # ==== 2. Radial scan หาขอบ inhibition zone ====
        th = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY_INV, 51, 2
        )

        radii = []
        for angle in range(0, 360, 5):  # ยิงทุก 5 องศา
            rad = np.deg2rad(angle)
            for dist in range(r, 300):  # เริ่มจากรัศมีเม็ดยาออกไป
                xx = int(x + dist * np.cos(rad))
                yy = int(y + dist * np.sin(rad))
                if xx < 0 or yy < 0 or xx >= th.shape[1] or yy >= th.shape[0]:
                    break
                if th[yy, xx] == 255:  # ชนขอบ clear zone
                    radii.append(dist)
                    break

        if len(radii) > 0:
            mean_radius = np.mean(radii)
            px_per_mm = (2 * r) / 6.0
            zone_mm = (2 * mean_radius) / px_per_mm

            # วาดวง inhibition zone โดยใช้รัศมีเฉลี่ย
            cv2.circle(img, (x, y), int(mean_radius), (255, 0, 0), 2)
            print(f"Drug center at ({x},{y}), Clear zone size: {zone_mm:.2f} mm")

# ==== Show result ====
cv2.imshow("Result", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


Drug center at (142,403), Clear zone size: 6.14 mm
Drug center at (896,160), Clear zone size: 6.12 mm
Drug center at (394,159), Clear zone size: 6.22 mm
Drug center at (148,159), Clear zone size: 6.15 mm
Drug center at (139,652), Clear zone size: 6.16 mm
Drug center at (393,406), Clear zone size: 6.02 mm
Drug center at (887,650), Clear zone size: 6.28 mm
Drug center at (643,649), Clear zone size: 6.19 mm
Drug center at (884,897), Clear zone size: 6.21 mm
Drug center at (633,897), Clear zone size: 6.22 mm
Drug center at (890,405), Clear zone size: 6.31 mm
Drug center at (389,649), Clear zone size: 6.23 mm
Drug center at (641,405), Clear zone size: 6.20 mm
Drug center at (134,898), Clear zone size: 6.07 mm


In [17]:
import cv2
import numpy as np
import math
import pandas as pd

IMAGE_PATH = "original.jpg"
DISK_DIAMETER_MM = 6.0   # เม็ดยามาตรฐาน

# โหลดภาพ
img = cv2.imread(IMAGE_PATH)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 5)

# ==== Step 1: Detect disks (ไม่ fix radius) ====
circles = cv2.HoughCircles(
    gray, cv2.HOUGH_GRADIENT, dp=1.2, minDist=40,
    param1=50, param2=50, minRadius=0, maxRadius=0
)

detected = []
if circles is not None:
    circles = np.round(circles[0, :]).astype("int")

    # กรองวงที่ไม่สมเหตุสมผล
    radii = circles[:, 2]
    median_r = np.median(radii)  # ใช้มัธยฐานช่วยกัน outlier
    filtered = [c for c in circles if 0.5*median_r < c[2] < 1.5*median_r]

    # เอาไม่เกิน 16 วง
    for (x, y, r) in filtered[:16]:
        detected.append((x, y, r))

# ==== Step 2: Radial scan clear zone ====
results = []
angles = np.arange(0, 360, 5)

for (x, y, r) in detected:
    radii = []
    for theta in angles:
        rad = math.radians(theta)
        dx, dy = math.cos(rad), math.sin(rad)

        dist = r
        while True:
            px = int(x + dx * dist)
            py = int(y + dy * dist)
            if px < 0 or py < 0 or px >= img.shape[1] or py >= img.shape[0]:
                break

            if gray[py, px] > 180:  # ขอบเขต clear zone
                break
            dist += 1

        radii.append(dist)

    clear_radius_px = min(radii) if radii else 0

    # คำนวณ px_per_mm จากรัศมีที่ detect ได้
    px_per_mm = (2 * r) / DISK_DIAMETER_MM
    clear_radius_mm = clear_radius_px / px_per_mm

    results.append([x, y, r, clear_radius_px, clear_radius_mm])

    cv2.circle(img, (x, y), r, (0, 255, 0), 2)
    cv2.circle(img, (x, y), int(clear_radius_px), (255, 0, 0), 2)
    cv2.circle(img, (x, y), 2, (0, 0, 255), 3)

# ==== Step 3: Export results ====
df = pd.DataFrame(results, columns=["x", "y", "radius_px", "clear_px", "clear_mm"])
df.to_csv("clear_zone_results.csv", index=False)

print(df)
cv2.imshow("Disks + Clear Zones", img)
cv2.waitKey(0)
cv2.destroyAllWindows()


      x    y  radius_px  clear_px  clear_mm
0   853  868        141       141  3.000000
1   135  137        114       131  3.447368
2   164  398        136       165  3.639706
3   865  597        146       159  3.267123
4   223  405        147       147  3.000000
5   143  189        127       128  3.023622
6   860  653        129       156  3.627907
7   633  885        133       133  3.000000
8   899  873        143       143  3.000000
9   431  154        146       146  3.000000
10  185  357        113       182  4.831858
11  895  166        148       148  3.000000
12  899  628        116       125  3.232759
13  164  439        132       165  3.750000
14  172  607        149       173  3.483221
15  152  658        145       153  3.165517
